# Import und Konfiguration
Importiere alle notwendigen Bibliotheken (pandas, numpy, yfinance, plotly, matplotlib, argparse etc.) und richte globale Variablen (wie HAS_IBKR, WORK_WITH_FILES, INTERVAL und WEEKLY) ein. Kommentiere, warum welche Konfiguration gewählt wurde.

In [ ]:
# Version 0.10
# Designed by T. Block
# Fall kopiert oder abgezeigt, der orginäre Wurzelpfad muss hier angegeben werden!
# License: GNU GENERAL PUBLIC LICENSE Version 3, 29 June 2007
#          Keine Gewährleistung oder Schadensansprüche, siehe  LICENSE Datei für weitere Informationen.
#          Die GNU Lizenz ist auch im Internet auf deutsch erhältlich.
#          https://www.gnu.org/licenses/gpl-3.0.html
#         
# Entferne das Kommentarzeichen, falls die Pakete installiert werden müssen
#!pip install --upgrade nbformat
#!pip install ipywidgets

In [ ]:
# Import und Konfiguration

# Importiere alle notwendigen Bibliotheken
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, date
import re, argparse
import pprint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
# filepath: cell in Ihrem Notebook
import plotly.io as pio
pio.renderers.default = 'vscode'  # 'browser' oder 'notebook_connected', vscode

# Konfiguration
# HAS_IBKR: Gibt an, ob die IBKR API verfügbar ist
# WORK_WITH_FILES: Bestimmt, ob Daten aus Dateien geladen werden sollen, um die Verarbeitung zu beschleunigen
# INTERVAL: Das Intervall für die Datenabfrage (z.B. '1d' für tägliche Daten)
# WEEKLY: Bestimmt, ob wöchentliche Daten verwendet werden sollen
global HAS_IBKR
try:
    import IbkrTws as ib
    HAS_IBKR = False
except:
    print("IbkrTws nicht verfügbar")
    HAS_IBKR = False

WORK_WITH_FILES = False  # Daten aus Dateien laden, um die Verarbeitung zu beschleunigen
INTERVAL = '1d'          # Intervall für die Datenabfrage: '1d', '1wk', '1mo'
WEEKLY = False           # Wöchentliche Daten verwenden

# Funktionen zur Datenbeschaffung
Definiere Funktionen wie fromNet, loadTickerData, saveToFile und appendNewToFile, um historische Aktienkurse von yfinance oder lokalen Dateien zu laden. Erkläre den Ablauf der Datenbeschaffung und den Umgang mit IBKR, wenn verfügbar.

In [2]:
# Funktionen zur Datenbeschaffung

def fromNet(symbol, prd, intrvl='1d'):
    """
    Lädt die Ticker-Daten von Yahoo Finance für den angegebenen Zeitraum.
    Wir verwenden das Intervall von 1 Tag.
    """
    df = pd.DataFrame()  # Leeres DataFrame
    # Historische Preisdaten anfordern
    # Gültige Intervalle: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
    df = yf.Ticker(symbol).history(period=prd, interval=intrvl)[['Open', 'Close', 'Low', 'High', 'Volume']]
    if df.empty:
        raise Exception(f"Keine Daten für {symbol} gefunden")
    return df

def loadTickerData(symbol, intrvl='1d'):
    """
    Lädt die Ticker-Daten aus dem Internet (WORK_WITH_FILES = False) oder aus einer Datei data/{symbol}.parquet.
    """
    df = pd.DataFrame()  # Leeres DataFrame
    if HAS_IBKR and Ibkr.isEnabled():
        df = Ibkr.get(symbol)
    elif not WORK_WITH_FILES:
        df = fromNet(symbol, '5y', intrvl)
    else:
        fileName = f'data/{symbol}.parquet'
        try:
            df = pd.read_parquet(fileName)
            last_save_date = df.index[-1].to_pydatetime()
            current_date = datetime.now()
            diff = current_date - last_save_date
            if diff.days > 1:
                dfNew = fromNet(symbol, f'{diff.days}d', intrvl)
                df = pd.concat([df, dfNew])
                df.to_parquet(fileName)
        except FileNotFoundError:
            print(f"Keine Datei für {symbol} gefunden")
            exit()
    df = df.sort_index()
    if WEEKLY:
        df = calcWeeklyFromDaily(df)
    return df

def saveToFile(ticker):
    """
    Speichert Ticker-Werte in der Datei data/{ticker}.parquet.
    """
    df = fromNet(ticker, '10y')
    fileName = f'data/{ticker}.parquet'
    df.to_parquet(fileName)
    return df

def appendNewToFile(symbol):
    """
    Lädt Ticker-Daten aus dem Internet und fügt neue Daten zur Datei data/{symbol}.parquet hinzu.
    """
    df = loadTickerData(symbol)
    last_save_date = df.index[-1]
    current_date = datetime.now()
    diff = current_date - last_save_date
    if diff.days > 0:
        dfNew = fromNet(symbol, f'{diff.days}d', '1d')
        df = pd.concat([df, dfNew]).drop_duplicates().reset_index(drop=True)
        fileName = f'data/{symbol}.parquet'
        df.to_parquet(fileName)
    return df

# Berechnung von Finanzkennzahlen und Indikatoren
Implementiere Funktionen zur Berechnung von Indikatoren wie Moving Averages, Bollinger Bands, MACD, RSI, Volatilität, Stochastic Oscillator und weiteren Finanzkennzahlen. Füge Erklärungen hinzu, die den Berechnungsprozess erläutern.

In [3]:
# Berechnung von Finanzkennzahlen und Indikatoren

# Moving Averages berechnen
def calcMovingAverages(df):
    """
    Berechnet die gleitenden Durchschnitte (Moving Averages) und fügt sie dem DataFrame hinzu.
    Fügt folgende Einträge zu df hinzu: Ma#, Cma#, Ema# mit # = 5, 10, 20, 50, 100, 200
    """
    averages = [5, 10, 20, 50, 100, 200]
    for avg in averages:
        df[f'Ma{avg}'] = df['Close'].rolling(window=avg).mean()
        df[f'Cma{avg}'] = df['Close'].expanding().mean()
        df[f'Ema{avg}'] = df['Close'].ewm(span=avg).mean()
    return df

# Bollinger Bands berechnen
def calcBollingerBands(df, n=20, m=2):
    """
    Berechnet die Bollinger Bands für das gegebene DataFrame.
    Fügt folgende Einträge zu df hinzu: Ma20, BbUpper, BbLower
    """
    m_avg = df['Close'].rolling(window=n).mean()
    m_std = df['Close'].rolling(window=n).std(ddof=0)
    df['BbUpper'] = m_avg + m * m_std
    df['BbLower'] = m_avg - m * m_std
    return df

# MACD berechnen
def calcMacd(df):
    """
    Berechnet den MACD (Moving Average Convergence Divergence) und fügt ihn dem DataFrame hinzu.
    Fügt folgende Einträge zu df hinzu: Macd, MacdSig, MacdDif
    """
    slow, fast, smooth = 26, 12, 9
    exp1 = df['Close'].ewm(span=fast, adjust=False).mean()
    exp2 = df['Close'].ewm(span=slow, adjust=False).mean()
    df['Macd'] = exp1 - exp2
    df['MacdSig'] = df['Macd'].ewm(span=smooth, adjust=False).mean()
    df['MacdDif'] = df['Macd'] - df['MacdSig']
    return df

# RSI berechnen
def calcRsi(df):
    """
    Berechnet den RSI (Relative Strength Index) und fügt ihn dem DataFrame hinzu.
    Fügt folgenden Eintrag zu df hinzu: RSI
    """
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=10, adjust=False).mean()
    ema_down = down.ewm(com=7, adjust=False).mean()
    rs = ema_up / ema_down
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Volatilität berechnen
def calcVolatility(df):
    """
    Berechnet die Volatilität der historischen Aktienkurse und fügt sie dem DataFrame hinzu.
    Fügt folgenden Eintrag zu df hinzu: Vola
    """
    df['Vola'] = np.log(df['Close'] / df['Close'].shift()).std() * 252 ** 0.5 * 100
    return df

# Stochastic Oscillator berechnen
def calcStochastic(df, periods=14, smooth=3):
    """
    Berechnet den Stochastic Oscillator und fügt ihn dem DataFrame hinzu.
    Fügt folgende Einträge zu df hinzu: stoch_k, stoch_k_signal, StMa, StMaS
    """
    high_roll = df['High'].rolling(periods).max()
    low_roll = df['Low'].rolling(periods).min()
    df['stoch_k'] = (df['Close'] - low_roll) / (high_roll - low_roll) * 100
    df['stoch_k_signal'] = df['stoch_k'].rolling(smooth).mean()
    df['StMa'] = df['stoch_k'].rolling(window=12).mean()
    df['StMaS'] = df['stoch_k_signal'].rolling(window=9).mean()
    return df

# Alle Indikatoren berechnen
def calculateIndicators(df):
    """
    Berechnet alle Indikatoren für das gegebene DataFrame.
    """
    df = calcMovingAverages(df)
    df = calcBollingerBands(df)
    df = calcMacd(df)
    df = calcRsi(df)
    df = calcVolatility(df)
    df = calcStochastic(df)
    return df

# Beispiel: Berechnung der Indikatoren für einen bestimmten Ticker
ticker = 'AAPL'
df = loadTickerData(ticker, INTERVAL)
df = calculateIndicators(df)
df.tail()  # Zeige die letzten Zeilen des DataFrames mit den berechneten Indikatoren

,Open,Close,Low,High,Volume,Ma5,Cma5,Ema5,Ma10,Cma10,...,BbLower,Macd,MacdSig,MacdDif,RSI,Vola,stoch_k,stoch_k_signal,StMa,StMaS
Date,,,,,,,,,,,,,,,,,,,,,
2025-03-17 00:00:00-04:00,213.309998,214.000000,209.970001,215.220001,48073400,214.997998,158.932100,216.665616,224.853999,158.932100,...,209.865890,-6.116742,-2.998154,-3.118588,27.938731,29.205099,15.262588,10.162086,25.495966,20.852976
2025-03-18 00:00:00-04:00,214.160004,212.690002,211.490005,215.149994,42432400,213.367999,158.974969,215.340411,222.530000,158.974969,...,207.078526,-6.587926,-3.716108,-2.871817,27.403585,29.205099,11.991026,13.148998,21.144342,18.404711
2025-03-19 00:00:00-04:00,214.220001,215.240005,213.750000,218.759995,54385400,213.020001,159.019802,215.306943,220.480000,159.019802,...,205.211929,-6.678591,-4.308605,-2.369986,33.063148,29.205099,19.151944,15.468519,18.968048,16.802991
2025-03-20 00:00:00-04:00,213.990005,214.100006,212.220001,217.490005,48862900,213.904004,159.063655,214.904631,218.357001,159.063655,...,203.571852,-6.764457,-4.799775,-1.964681,32.384489,29.205099,15.950598,15.697856,17.885650,14.543912
2025-03-21 00:00:00-04:00,211.559998,218.270004,211.279999,218.839996,93954500,214.860004,159.110757,216.026422,216.277000,159.110757,...,202.893742,-6.421993,-5.124219,-1.297774,41.192310,29.205099,29.893800,21.665447,17.764857,13.640413


# Implementierung der Strategie
Definiere Funktionen, die Kauf- und Verkaufssignale anhand der berechneten Indikatoren identifizieren (z.B. isBuySignal, isSellSignal, implementStrategy). Kommentiere den Entscheidungsprozess für die Signale.

In [4]:
# Implementierung der Strategie

# Funktion zur Identifikation von Kaufsignalen
def isBuySignal(df, i):
    """
    Identifiziert Kaufsignale basierend auf verschiedenen Indikatoren.
    """
    # Beispielhafte Bedingungen für ein Kaufsignal
    conditions = [
        df['stoch_k'].iloc[i] < 20,  # Stochastic Oscillator unter 20
        df['RSI'].iloc[i] < 30,      # RSI unter 30
        df['MacdDif'].iloc[i] > 0    # MACD-Differenz positiv
    ]
    return all(conditions)

# Funktion zur Identifikation von Verkaufssignalen
def isSellSignal(df, i):
    """
    Identifiziert Verkaufssignale basierend auf verschiedenen Indikatoren.
    """
    # Beispielhafte Bedingungen für ein Verkaufssignal
    conditions = [
        df['stoch_k'].iloc[i] > 80,  # Stochastic Oscillator über 80
        df['RSI'].iloc[i] > 70,      # RSI über 70
        df['MacdDif'].iloc[i] < 0    # MACD-Differenz negativ
    ]
    return all(conditions)

# Funktion zur Implementierung der Strategie
def implementStrategy(df):
    """
    Markiert die Kauf- und Verkaufssignale für die gegebenen Ticker-Werte im DataFrame.
    Fügt folgende Einträge zu df hinzu: sigBuy, sigSell
    """
    buy_price, sell_price = [], []
    for i in range(len(df)):
        if isBuySignal(df, i):
            buy_price.append(df['Close'].iloc[i])
            sell_price.append(np.nan)
        elif isSellSignal(df, i):
            buy_price.append(np.nan)
            sell_price.append(df['Close'].iloc[i])
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
    df['sigBuy'] = buy_price
    df['sigSell'] = sell_price
    return df

# Beispiel: Implementierung der Strategie für einen bestimmten Ticker
df = implementStrategy(df)
df.tail()  # Zeige die letzten Zeilen des DataFrames mit den Kauf- und Verkaufssignalen

,Open,Close,Low,High,Volume,Ma5,Cma5,Ema5,Ma10,Cma10,...,MacdSig,MacdDif,RSI,Vola,stoch_k,stoch_k_signal,StMa,StMaS,sigBuy,sigSell
Date,,,,,,,,,,,,,,,,,,,,,
2025-03-17 00:00:00-04:00,213.309998,214.000000,209.970001,215.220001,48073400,214.997998,158.932100,216.665616,224.853999,158.932100,...,-2.998154,-3.118588,27.938731,29.205099,15.262588,10.162086,25.495966,20.852976,NaN,NaN
2025-03-18 00:00:00-04:00,214.160004,212.690002,211.490005,215.149994,42432400,213.367999,158.974969,215.340411,222.530000,158.974969,...,-3.716108,-2.871817,27.403585,29.205099,11.991026,13.148998,21.144342,18.404711,NaN,NaN
2025-03-19 00:00:00-04:00,214.220001,215.240005,213.750000,218.759995,54385400,213.020001,159.019802,215.306943,220.480000,159.019802,...,-4.308605,-2.369986,33.063148,29.205099,19.151944,15.468519,18.968048,16.802991,NaN,NaN
2025-03-20 00:00:00-04:00,213.990005,214.100006,212.220001,217.490005,48862900,213.904004,159.063655,214.904631,218.357001,159.063655,...,-4.799775,-1.964681,32.384489,29.205099,15.950598,15.697856,17.885650,14.543912,NaN,NaN
2025-03-21 00:00:00-04:00,211.559998,218.270004,211.279999,218.839996,93954500,214.860004,159.110757,216.026422,216.277000,159.110757,...,-5.124219,-1.297774,41.192310,29.205099,29.893800,21.665447,17.764857,13.640413,NaN,NaN


# Visualisierung der Ergebnisse
Erstelle Plotfunktionen (z.B. plotStrategy, plotStatistics) zur Visualisierung der Aktienkurse, Indikatoren und identifizierten Signale. Nutze Plotly und Matplotlib für interaktive und statische Diagramme und kommentiere die Visualisierungsschritte.

In [5]:
HAS_LT_SIG_BUY = False
def plotStrategy(dataFrame, ticker):
  """Plots the strategy for ticker given in dataFrame."""
  fig, ax = plt.subplots(figsize=(16, 10))
  ax.plot(dataFrame['Close']  ,      label=ticker, linewidth=2)
  ax.plot(dataFrame['sigBuy'] , 'o', label='Buy Signal', markersize=10, color='green')
  ax.plot(dataFrame['sigSell'], 'o', label='Sell Signal', markersize=10, color='red')
  if HAS_LT_SIG_BUY:
    ax.plot(dataFrame['ltSigBuy'], 'o', label='ltBuy Signal', markersize=10, color='white')
    ax.plot(dataFrame['ltSigSell'], 'o', label='ltSell Signal', markersize=10, color='black')
  ax.legend(loc='upper left')
  plt.show()
#------------------------------------------------------------------------------
def addPriceChart(fig, xData, plotData, curRow):
  """Add the price candlestick and markers to the chart."""
  #--- Plot the price in candlestick format.
  fig.add_trace(go.Candlestick(
    x=xData, name="Candlestick",
    open=plotData['Open'], high=plotData['High'], low=plotData['Low'], close=plotData['Close']    
  ), row=curRow, col=1)
  #--- add buy and sell markers
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['sigBuy'], name="buy", 
    mode='markers', marker_color='cyan', marker_size=10
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['sigSell'], name="sell",
    mode='markers', marker_color='yellow', marker_size=10
  ), row=curRow, col=1)
  if HAS_LT_SIG_BUY:
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['ltSigBuy'], name="ltBuy",
      mode='markers', marker_color='blue', marker_size=7
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['ltSigSell'], name="ltSell",
      mode='markers', marker_color='black', marker_size=7
    ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def addBollingerBands(fig, xData, plotData, curRow):
  """Add the Bollinger bands to the chart."""
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['BbUpper'], name="Upper Band",
    line=dict(color='black', width=1)
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ma5'], name='Ma 5',
    line=dict(color='gray', width=1), opacity=1.0
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['BbLower'], name="Lower Band",
    line=dict(color='black', width=1)
  ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------

def addMovingAverages(fig, xData, plotData, curRow):
  """Add moving averages to the chart."""
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ema10'], name='Ema 10',
    line=dict(color='red', width=0.1), opacity=0.7
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ema20'], name='Ema 20',
    line=dict(color='orange', width=1), opacity=0.7
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ema50'], name='Ema 50',
    line=dict(color='blue', width=1), opacity=0.7
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ema100'], name='Ema 100',
    line=dict(color='pink', width=1)
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['Ema200'], name='Ema 200',
    line=dict(color='skyblue', width=1)
  ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def addMaxText(fig, dataFrame, xData, plotData, startIndex, curRow):
  """Add maximum value text marker to chart."""
  maxValue = dataFrame['High'].max()
  textPosY = dataFrame['Low'].iloc[startIndex:].min() - 5
  fig.add_trace(go.Scatter(
    x=[xData[0]], y=[textPosY],
    mode="lines+markers+text", name="Lines, Markers and Text",
    text=[f"max={maxValue:.2f}"],
    textposition="top center"
  ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def addStochastic(fig, xData, plotData, curRow):
  """Add stochastic indicators to the chart."""
  if 0:
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['Prediction'], name="Prediction",
      line=dict(color='black', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData,
      y=plotData['PredictionMa'], name="Prediction",
      line=dict(color='black', width=1)
    ), row=curRow, col=1)
  else:
    # fig.add_trace(go.Bar(x=xData, y=plotData['Volume']), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['stoch_k'], name='Stochastic',
      line=dict(color='red', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData,
      y=plotData['stoch_k_signal'], name='StochasticSignal',
      line=dict(color='aquamarine', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['StMa'], name='StMa',
      line=dict(color='green', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=xData, y=plotData['StMaS'], name='StMaS',
      line=dict(color='blue', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=[xData[0], xData[-1]], y=[20, 20], name="b20",
      line=dict(color='red', width=1)
    ), row=curRow, col=1)
    fig.add_trace(go.Scatter(
      x=[xData[0], xData[-1]], y=[80, 80], name="b80",
      line=dict(color='green', width=1)
    ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def addMacd(fig, xData, plotData, curRow):
  """Add MACD indicators to the chart."""
  yValue = plotData['MacdDif']
  colors = np.where(yValue < 0, 'red', 'green')
  fig.add_trace(go.Bar(
    x=xData, y=yValue, name="MacdDif",
    marker_color=colors, opacity=1
  ), row=curRow, col=1)
  # fig.add_trace(go.Scatter(x=xData, y=plotData['MacdSig'], line=dict(color='blue', width=0.5),
  #               name='MacdSig'), row=curRow, col=1)
  # fig.add_trace(go.Scatter(x=xData, y=plotData['Macd'], line=dict(color='blue', width=0.5),
  #               name='Macd'), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def addRsi(fig, xData, plotData, curRow):
  """Add RSI indicator to the chart."""
  fig.add_trace(go.Scatter(
    x=xData, y=plotData['RSI'], name="RSI",
    marker_color='black', line=dict(width=1)
  ), row=curRow, col=1)
  # fig.add_trace(go.Scatter(x=xData, y=plotData['Vola'], name="Vola",
  #               marker_color='yellow', line=dict(width=1)), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=[xData[0], xData[-1]], y=[30, 30], name="n30",
    marker_color='red', line=dict(width=1)
  ), row=curRow, col=1)
  fig.add_trace(go.Scatter(
    x=[xData[0], xData[-1]], y=[70, 70], name="b70",
    marker_color='green', line=dict(width=1)
  ), row=curRow, col=1)
  return curRow
#------------------------------------------------------------------------------
def updateXAxes(dataFrame, fig):
  """Hide dates with no values, removing all empty dates."""
  if INTERVAL == '1d':
    dtAll = pd.date_range(start=dataFrame.index[0], end=dataFrame.index[-1])
    dtObs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(dataFrame.index)]
    dtBreaks = [d for d in dtAll.strftime("%Y-%m-%d").tolist() if d not in dtObs]
    fig.update_xaxes(rangebreaks=[dict(values=dtBreaks)])
#------------------------------------------------------------------------------
def updateLayout(fig, ticker):
  """Modify default layout of the chart."""
  fig.update_layout(
    autosize=True,
    showlegend=True,
    xaxis_rangeslider_visible=False,
    xaxis=dict(zerolinecolor='black', showticklabels=False),
    paper_bgcolor="white",
    height=900,
    title_text=f"Stock Price and Indicators of ticker '{ticker}'"
  )
#------------------------------------------------------------------------------
def plotStatistics(dataFrame, ticker, numberOfDaysToShow=365*3):
  # Inspired by https://python.plainenglish.io/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682
  # https://medium.com/codex/algorithmic-trading-with-macd-in-python-1c2769a6ad1b
  if len(dataFrame) < numberOfDaysToShow:
    numberOfDaysToShow = len(dataFrame)
  startIndex = len(dataFrame) - numberOfDaysToShow
  xData = dataFrame.index[startIndex:]
  plotData = dataFrame[startIndex:]
  fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    row_heights=[0.65, 0.15, 0.1, 0.1],
    vertical_spacing=0.035,
    column_widths=[1],
    subplot_titles=(f"Price {ticker}", "Stochastic", "Macd", "RSI")
  )
  curRow = 1
  curRow = addPriceChart(fig, xData, plotData, curRow)
  curRow = addBollingerBands(fig, xData, plotData, curRow)
  curRow = addMovingAverages(fig, xData, plotData, curRow)
  curRow = addMaxText(fig, dataFrame, xData, plotData, startIndex, curRow)
  #--- Add Stochastic
  curRow += 1
  curRow = addStochastic(fig, xData, plotData, curRow)
  #--- Add MACD
  curRow += 1
  curRow = addMacd(fig, xData, plotData, curRow)
  #--- Add RSI
  curRow += 1
  curRow = addRsi(fig, xData, plotData, curRow)
  updateXAxes(dataFrame, fig)
  updateLayout(fig, ticker)
  fig.show()

#plotStatistics(df, ticker)

# Interaktive Aktienauswahl und Anzeige
Integriere interaktive Elemente (z.B. ipywidgets) um einen Aktien-Ticker auszuwählen. Implementiere eine Schnittstelle, die basierend auf der Auswahl die zugehörige Analyse (Laden der Daten, Berechnung der Indikatoren, Strategieumsetzung und Plotting) durchführt, und erkläre, wie die Interaktivität funktioniert.

In [6]:
from ipywidgets import interact

def show_analysis(ticker):
    df = loadTickerData(ticker, INTERVAL)
    df = calculateIndicators(df)
    df = implementStrategy(df)
    plotStatistics(df, ticker)

interact(show_analysis, ticker=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META','SPY', 'vow3.de'])

interactive(children=(Dropdown(description='ticker', options=('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'SPY', …

<function __main__.show_analysis(ticker)>